# Question 3

In [ ]:
!pip install nltk==3.5
!pip install python-crfsuite

In [ ]:
from nltk.corpus import treebank
from nltk.tag.api import TaggerI
from nltk.tag import CRFTagger, untag, RegexpTagger
import unicodedata
import re
import nltk
from nltk.tag import brill, brill_trainer 

nltk.download('treebank')

[nltk_data] Downloading package treebank to /home/jovyan/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

# CRFTagger

In [ ]:
# call the class
ct = CRFTagger()
# create the train set et test set 
train_set = treebank.tagged_sents()[:3000]
test_set = treebank.tagged_sents()[3000:]

In [ ]:
# train the model
#ct.train(train_set,'model.crf.tagger')
# to create the tags
# ct.tag_sents([['dog','is','good'], ['Cat','eat','meat']])
# measure accuracy I guess?
#ct.evaluate(test_set)

# save the model
#ct.set_model_file('model.crf.tagger')

### Look into the words before or after

do we modify the _get_features method here https://www.nltk.org/_modules/nltk/tag/crf.html and create a new class ?

In [ ]:
#from crftagger import CRFTagger_modified
# call the class
#modified_crf = CRFTagger_modified()
# train the model
#modified_crf.train(train_set[:10],'model.crf.tagger')

In [ ]:
def get_features(self,tokens, idx, lag = 0):
        """
        Extract basic features about this word including
             - Current Word
             - Is Capitalized ?
             - Has Punctuation ?
             - Has Number ?
             - Suffixes up to length 3
        Note that : we might include feature over previous word, next word ect.

        :return : a list which contains the features
        :rtype : list(str)

        """
        token = tokens[idx]

        feature_list = []
        side = ''
        side = '_PREV_' if lag > 0  else 'AFT_' if lag < 0 else ''

        if not token:
            return feature_list

        # print(token)

        # Capitalization
        if token[0].isupper():
            feature_list.append("CAPITALIZATION" + side)

        # Number
        if(lag == 0):
            if re.search(self._pattern, token) is not None:
                feature_list.append("HAS_NUM" + side)

        # Punctuation
        if(lag == 0):
            punc_cat = set(["Pc", "Pd", "Ps", "Pe", "Pi", "Pf", "Po"])
            if all(unicodedata.category(x) in punc_cat for x in token):
                feature_list.append("PUNCTUATION" + side)

        # Suffix up to length 3
        if len(token) > 1:
            feature_list.append("SUF_"+ side + token[-1:])
        if len(token) > 2:
            feature_list.append("SUF_"+ side + token[-2:])
        # still get 94% without 
        if len(token) > 3:
            feature_list.append("SUF_"+ side + token[-3:])

        #if idx>=1 and lag < 1:
            #print(get_features(self,tokens, idx -1,1))
            #feature_list = get_features(self,tokens, idx -1, 1) + feature_list
            #feature_list.append("PREV1_" + tokens[idx-1])
        if idx==1 and lag < 1:
            feature_list.append("PREV1_" + tokens[idx])

        if idx==len(tokens)-2 and lag < 1:
            feature_list.append("NEXT1_" + tokens[idx])

        if idx<len(tokens)-2 and lag < 1:
            feature_list.append("NEXT1_" + tokens[idx+1])
            feature_list.append("NEXT2_" + tokens[idx+2])
            #feature_list.append("NEXT3_" + tokens[idx+3])

        if idx>=2 and lag < 1:
            feature_list.append("PREV1_" + tokens[idx-1])
            feature_list.append("PREV2_" + tokens[idx-2])
            #feature_list.append("PREV3_" + tokens[idx-3])
        
        #if idx>=3 and lag < 1:
        #    feature_list.append("PREV3_" + tokens[idx-3])

        #if idx>2 and lag < 1:   
        if idx < len(tokens)-1 and lag < 1:
            # print(get_features(self,tokens, idx -1,1))
            feature_list = get_features(self,tokens, idx -1, 1) + feature_list
            #feature_list.append("NEXT_" + tokens[idx+1])
            #print(feature_list)

        feature_list.append("WORD_" + side + token)
        
        return feature_list

In [ ]:
from functools import partial
# call the class
crf = CRFTagger()
crf._feature_func = partial(get_features,crf)
# train the model
# crf.train(train_set[:10],'model.crf.tagger')
crf.train(train_set, 'model.crf.tagger')
crf.evaluate(test_set)


0.9526009065400388

# BrillTagger

In [ ]:
from nltk.tag.brill_trainer import BrillTaggerTrainer
from nltk.tbl.template import Template
from nltk.tbl.template import Template
from nltk.tag.brill import Pos, Word
from nltk.tag import untag, RegexpTagger, BrillTaggerTrainer

In [ ]:
brill.Template(brill.Pos([-1]))

Template(Pos([-1]))

In [ ]:
backoff = RegexpTagger([
    (r'^-?[0-9]+(.[0-9]+)?$', 'CD'),   # cardinal numbers
    (r'(The|the|A|a|An|an)$', 'AT'),   # articles
    (r'.*able$', 'JJ'),                # adjectives
    (r'.*ness$', 'NN'),                # nouns formed from adjectives
    (r'.*ly$', 'RB'),                  # adverbs
    (r'.*s$', 'NNS'),                  # plural nouns
    (r'.*ing$', 'VBG'),                # gerunds
    (r'.*ed$', 'VBD'),                 # past tense verbs
    (r'.*', 'NN')                      # nouns (default)
    ])
    
baseline = backoff

In [ ]:
Template._cleartemplates() #clear any templates created in earlier tests
templates = [Template(Pos([-1])), Template(Pos([-1]), Word([0]))]

In [ ]:
tt = BrillTaggerTrainer(baseline, templates, trace=3)
tagger1 = tt.train(train_set, max_rules=10)

TBL train (fast) (seqs: 3000; tokens: 77511; tpls: 2; min score: 2; min acc: None)
Finding initial useful rules...
    Found 9476 useful rules.

           B      |
   S   F   r   O  |        Score = Fixed - Broken
   c   i   o   t  |  R     Fixed = num tags changed incorrect -> correct
   o   x   k   h  |  u     Broken = num tags changed correct -> incorrect
   r   e   e   e  |  l     Other = num tags changed incorrect -> incorrect
   e   d   n   r  |  e
------------------+-------------------------------------------------------
38973897   0  13  | AT->DT if Pos:NN@[-1]
27042704   0   0  | NN->, if Pos:NN@[-1] & Word:,@[0]
20392039   0   0  | NN->. if Pos:NN@[-1] & Word:.@[0]
13651365   0   0  | NN->IN if Pos:NN@[-1] & Word:of@[0]
13541354   0   3  | NN->TO if Pos:NN@[-1] & Word:to@[0]
 895 920  25 154  | NN->VB if Pos:TO@[-1]
 802 803   1   6  | NN->IN if Pos:NN@[-1] & Word:in@[0]
 717 719   2   4  | NN->CC if Pos:NN@[-1] & Word:and@[0]
 714 714   0   0  | NN->, if Pos:NNS@[-1] & Word

In [ ]:
tagger1.rules()[1:3]

(Rule('001', 'NN', ',', [(Pos([-1]),'NN'), (Word([0]),',')]),
 Rule('001', 'NN', '.', [(Pos([-1]),'NN'), (Word([0]),'.')]))

In [ ]:
train_stats = tagger1.train_stats()
[train_stats[stat] for stat in ['initialerrors', 'finalerrors', 'rulescores']]

[59527, 44355, [3897, 2704, 2039, 1365, 1354, 895, 802, 717, 714, 685]]

In [ ]:
tagger1.print_template_statistics(printunused=False)

TEMPLATE STATISTICS (TRAIN)  2 templates, 10 rules)
TRAIN (  77511 tokens) initial 59527 0.2320 final: 44355 0.4278 
#ID | Score (train) |  #Rules     | Template
--------------------------------------------
001 |  9695   0.639 |   7   0.700 | Template(Pos([-1]),Word([0]))
000 |  5477   0.361 |   3   0.300 | Template(Pos([-1]))




In [ ]:
tagger1.evaluate(test_set)

0.45750053960716597

In [ ]:
#templates = nltk.tag.brill.brill24()
#tt = BrillTaggerTrainer(train_set, templates, trace=1)
#tagger1 = tt.train(train_set, max_rules=100)

# Question 5

In [ ]:
!pip install spacy
!python3 -m spacy download en_core_web_sm
!python3 -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
#!python3 -m spacy download en_core_web_sm
!python3 -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
from nltk.tag.api import TaggerI
import spacy
#spacy.load('en_core_web_sm')
spacy.load('en_core_web_lg')

OSError: [E050] Can't find model 'en_core_web_lg'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

https://www.nltk.org/_modules/nltk/tag/api.html#TaggerI

https://www.nltk.org/_modules/nltk/tag/crf.html#CRFTagger

In [ ]:
from abc import ABCMeta, abstractmethod
from itertools import chain


from nltk.internals import overridden
from nltk.metrics import accuracy
from nltk.tag.util import untag
import spacy
import en_core_web_lg
import en_core_web_sm


class PersonalTagger(TaggerI):

     
    # try also  the _lg one
    

    def __init__(self, feature_func=None, verbose=False, training_opt={}):

        #spacy.load('en_core_web_sm')
        #spacy.load('en_core_web_lg')

        import pycrfsuite
        self._tagger = pycrfsuite.Tagger()

        self._verbose = verbose
        self._training_options = training_opt
        self._pattern = re.compile(r"\d")
        self.model = "en_core_web_lg"
        self.nlp = en_core_web_lg.load()
        #self.nlp = spacy.load(self.model,disable=["parser", "ner"])
        

    def tag_sents(self, sents):

        # We need the list of sentences instead of the list generator for matching the input and output
        result = []

        
        for sent in sents:
            doc = spacy.tokens.doc.Doc(self.nlp.vocab, words=sent)

            for _, proc in self.nlp.pipeline:
                doc = proc(doc)

            labels = []

            for word in doc:
                labels.append(word.tag_)

            tagged_sent = list(zip(sent, labels))

            result.append(tagged_sent)
            

        return result

    def tag(self, tokens):
        return self.tag_sents([tokens])[0]

    def evaluate(self, gold):

        tagged_sents = self.tag_sents(untag(sent) for sent in gold)
        gold_tokens = list(chain(*gold))
        test_tokens = list(chain(*tagged_sents))
        return accuracy(gold_tokens, test_tokens)

In [ ]:
from nltk.corpus import treebank
train_set = treebank.tagged_sents()[:3000]
test_set = treebank.tagged_sents()[3000:]
whole_set = treebank.tagged_sents()

In [ ]:
personaltagger = PersonalTagger()

personaltagger.evaluate(whole_set)

0.8937383288966586

In [ ]:
import spacy
model = "en_core_web_sm" 

# try also  the _lg one
nlp = spacy.load(model,disable=["parser", "ner"])

# to go faster
# we want to do this:
# doc = nlp(’hello world !’)
# but the tokenization would change from the one in treebank
# which would cause problems with the function evaluate
# so instead  do this more convoluted thing:


tokens_of_my_sentence = ['hello', 'world', '!']
doc = spacy.tokens.doc.Doc(nlp.vocab, words=tokens_of_my_sentence)

for _, proc in nlp.pipeline:
    print(proc)
    doc = proc(doc)

print(doc)
# now doc is ready:
for t in doc:
    print(f'{t.text:20s} {t.tag_}')

hello world ! 
hello                UH
world                NNP
!                    .


In [ ]:
from nltk.tag.brill_trainer import BrillTaggerTrainer
from nltk.tbl.template import Template
from nltk.tbl.template import Template
from nltk.tag.brill import Pos, Word
from nltk.tag import untag, RegexpTagger, BrillTaggerTrainer

In [ ]:
backoff = RegexpTagger([
    (r'^-?[0-9]+(.[0-9]+)?$', 'CD'),   # cardinal numbers
    (r'(The|the|A|a|An|an)$', 'AT'),   # articles
    (r'.*able$', 'JJ'),                # adjectives
    (r'.*ness$', 'NN'),                # nouns formed from adjectives
    (r'.*ly$', 'RB'),                  # adverbs
    (r'.*s$', 'NNS'),                  # plural nouns
    (r'.*ing$', 'VBG'),                # gerunds
    (r'.*ed$', 'VBD'),                 # past tense verbs
    (r'.*', 'NN')                      # nouns (default)
    ])
    
baseline = backoff

Template._cleartemplates() #clear any templates created in earlier tests
templates = [Template(Pos([-1])), Template(Pos([-1]), Word([0]))]
templates2 = nltk.tag.brill.brill24()

In [ ]:
tt = BrillTaggerTrainer(personaltagger, templates2)
tagger = tt.train(train_set)

In [ ]:
tagger.evaluate(test_set)

0.9721562702352687

In [ ]:
crf = CRFTagger()
crf.train(train_set, 'model.crf.tagger')
tt = BrillTaggerTrainer(crf, templates2)
tagger2 = tt.train(train_set)

In [ ]:
tagger2.evaluate(test_set)

0.9509173321821713